# EDO: Problema de valor de contorno
## Método do tiro

Assuma a seguinte EDO de segunda ordem que representa a distribuição de calor ao longo de uma barra

\begin{align}
\frac{d^2T}{dx^2} &= k_0 \left(T(x) - T_s\right) + k_1 \left(T^4(x) - T_s^4\right) \\
T(0)   &= 493 \text{K}, ~T(0.1) = 293 \text{K}\\
\end{align}
onde $k = [167, ~9.45\times 10^{-08}]$ são parâmetros do sistema, $T_s=293$K é a temperatura ambiente, $T(x)$ é a Temperatura em função do comprimento, $x$, da barra; $T(0)$ e $T(0.1)$ são as condições de contorno.

### Redução de ordem

Para resolvermos a EDO-PVC, primeiramente vamos transformá-la em um problema PVI, dessa forma é preciso fazermos a redução de ordem

\begin{align}
\frac{dT(x)}{dx} &= w\\
\frac{dw(x)}{dx} &= k_0 \left(T(x) - T_s\right) + k_1 \left(T^4(x) - T_s^4\right)
\end{align}

Feito isso, para resolver o sistema de primeira ordem encontrado se faz nescessário as condições iniciais, $T(0)$ e $w(0)$. Porém só conhecemos a condição inicial $T(0) = 493$K, e por isso precisaremos chutar uma condição para $w(0)=?$

A partir daí resolvemos o sistema de EDO por um método PVI

In [ ]:
#@title Visualização
%%html
<p align="center">
<iframe src="https://www.geogebra.org/classic/tnewvaxp?embed"
        width="1200" height="700"
        allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>
</p>



Observe que pelo gráfico, que, para um valor de $w(0) = -1550$, o ultimo valor encontrado pela solução PVI, $T_{pvi}(0.1) = 443.5$, é diferente da condição de contorno dada, $T(0.1) = 293$,

Sendo assim temos que mudar o valor de $w(0)$, até fazermos $T_{pvi}(0.1) = T(0.1)$

Sendo assim, programe o método do tiro para resolução da EDO apresentada. Compare a solução da EDO para o $w(0)$ chutado com a solução encontrada pelo método do tiro



Equipe:
* Eduardo Fontes
* Dyego Muritiba
* Danyllo Muritiba

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
# Redução de ordem da EDO
def edo(x, Y):
    k = [167.0, 9.45e-8]  # Constantes do sistema
    T_s = 293  # Temperatura ambiente
    T = Y[0]  # Temperatura
    w = Y[1]  # Calor

    # EDOs resultantes
    dTdx = w
    dwdx = k[0] * (T - T_s) + k[1] * (T**4 - T_s**4)
    return [dTdx, dwdx]

# Função para resolver o problema de valor inicial (PVI) dado um chute inicial para w(0)
def solve_pvi(w0):
    T0 = 493.0  # Condição inicial para T
    x_span = (0, 0.1)  # Intervalo para x
    y0 = [T0, w0]  # Condições iniciais

    sol = solve_ivp(edo, x_span, y0, t_eval=np.linspace(0, 0.1, 100))
    return sol

# Função de erro para ajustar a condição inicial w(0)
def error_w0(w0):
    sol = solve_pvi(w0)
    T_final = sol.y[0, -1]  # Valor de T no ponto final do intervalo
    return T_final - 293.0  # Condição de contorno desejada T(0.1) = 293

# Derivada numérica da função de erro usando diferenças finitas
def derivative_error_w0(w0, epsilon=1e-5):
    return (error_w0(w0 + epsilon) - error_w0(w0 - epsilon)) / (2 * epsilon)

# Método de Newton-Raphson para encontrar o w0 adequado
def newton_raphson(initial_guess, tol=1e-5, max_iter=100):
    w0 = initial_guess
    for i in range(max_iter):
        f_w0 = error_w0(w0)
        df_w0 = derivative_error_w0(w0)
        if abs(f_w0) < tol:
            print(f'Convergiu na iteração número: {i}')
            return w0
        w0 -= f_w0 / df_w0

# Chute inicial para w0
chuteInicial = 334.0

# Encontrar o w0 ótimo usando Newton-Raphson
w0_opt = newton_raphson(chuteInicial)

# Resolver o sistema com o valor encontrado de w0
sol_opt = solve_pvi(w0_opt)

# Plotar a solução encontrada
plt.plot(sol_opt.t, sol_opt.y[0], label='Método de Newton-Raphson')
plt.axhline(y=293, color='r', linestyle='--', label='Cond. Contorno T(0.1)=293')
plt.xlabel('x (m)')
plt.ylabel('T (K)')
plt.title('Distribuição de Temperatura na Barra')
plt.legend()
plt.show()
